# Ejercicio 2: Reddit API Data Collection & Sentiment Analysis

**Autora:** Bianca Peraltilla  
**Curso:** Python for Data Science (UP, 2025-II)  

---

## Objetivo
- Conectarme a la API de Reddit usando **PRAW**.  
- Extraer publicaciones de subreddits políticos:  
  - r/politics  
  - r/PoliticalDiscussion  
  - r/worldnews  
- Guardar la información principal de las publicaciones:  
  - título, score (upvotes), número de comentarios, id y url.  
- Extraer comentarios de los posts más relevantes (5 por post).  
- Almacenar los datos en DataFrames de **pandas** para posterior análisis de sentimiento.  

---

## Paso 1: Preparación del entorno
En este paso:
1. Instalo y cargo las librerías necesarias:  
   - **praw** → conexión con la API de Reddit.  
   - **python-dotenv** → gestión de credenciales de manera segura.  
   - **pandas** → manejo y almacenamiento de datos.  
2. Verifico que todas las librerías se importan correctamente.  

In [15]:
%pip install --quiet praw python-dotenv pandas


Note: you may need to restart the kernel to use updated packages.


In [16]:
import praw
import pandas as pd
from dotenv import load_dotenv

print("PRAW:", praw.__version__)
print("Pandas:", pd.__version__)
print("dotenv listo ✔")


PRAW: 7.8.1
Pandas: 2.2.2
dotenv listo ✔


In [17]:
try:
    import praw, pandas as pd
    from dotenv import load_dotenv
    print("✅ Todo OK, puedes seguir con Reddit.")
except Exception as e:
    print("❌ Falta algo:", e)


✅ Todo OK, puedes seguir con Reddit.


## Paso 2: Configuración de credenciales y conexión con la API de Reddit

En este paso:  
1. Creo un archivo `.env` que guarda mis credenciales de forma segura.  
2. Cargo estas credenciales en el notebook con **python-dotenv**.  
3. Establezco la conexión con la API de Reddit usando **PRAW**.  


In [24]:
import os
from dotenv import load_dotenv
import praw

# Cargar variables desde el archivo .env
load_dotenv()

# Conexión a Reddit usando PRAW
reddit = praw.Reddit(
    client_id=os.getenv("REDDIT_CLIENT_ID"),
    client_secret=os.getenv("REDDIT_CLIENT_SECRET"),
    user_agent=os.getenv("REDDIT_USER_AGENT"),
    username=os.getenv("REDDIT_USERNAME"),
    password=os.getenv("REDDIT_PASSWORD")
)

# Prueba de conexión: verifico mi identidad
print("✅ Conexión exitosa con Reddit")
print("Usuario autenticado:", reddit.user.me())


✅ Conexión exitosa con Reddit
Usuario autenticado: Positive-Fly6703


## Parte 2: Recolección de posts por subreddit

**Objetivo:** descargar **20 posts** de cada subreddit político usando **PRAW** y guardar los resultados en un CSV para análisis posterior.

- Subreddits objetivo:
  - `r/politics`
  - `r/PoliticalDiscussion`
  - `r/worldnews`
- Tipo de listado: dejo **parametrizable** `hot` o `top` (inicio con `hot`).
- Campos extraídos por post:
  - `title`, `score` (upvotes), `num_comments`, `id`, `url`, `subreddit`
- Salida: `output/reddit_posts.csv`

> Nota: este dataset será la base para extraer comentarios (siguiente paso) y luego hacer análisis de sentimiento.


In [ ]:
# Paso 3 — Descarga de 20 posts por subreddit y guardado en CSV

import time
from pathlib import Path
import pandas as pd

# 1) Defino parámetros (puedo cambiar "hot" por "top" si quiero)
subreddits = ["politics", "PoliticalDiscussion", "worldnews"]
list_type = "hot"     # opciones: "hot" o "top"
limit_per_sub = 20    # posts por subreddit

# 2) Aseguro carpeta de salida
OUTDIR = Path("output")
OUTDIR.mkdir(exist_ok=True)

def fetch_posts_df(reddit, subreddit: str, list_type: str = "hot", limit: int = 20) -> pd.DataFrame:
    """
    Descargo posts de un subreddit y devuelvo un DataFrame limpio
    con las columnas que voy a analizar después.
    """
    sr = reddit.subreddit(subreddit)
    it = sr.hot(limit=limit) if list_type == "hot" else sr.top(limit=limit)

    rows = []
    for p in it:
        rows.append({
            "subreddit": subreddit,
            "id": p.id,
            "title": p.title or "",
            "score": int(getattr(p, "score", 0) or 0),
            "num_comments": int(getattr(p, "num_comments", 0) or 0),
            "url": p.url or ""
        })
    df = pd.DataFrame(rows).drop_duplicates(subset=["id"]).reset_index(drop=True)
    return df

# 3) Recojo 20 por cada subreddit, concateno y guardo
frames = []
for s in subreddits:
    print(f"Descargando {limit_per_sub} posts ({list_type}) de r/{s}…")
    df_s = fetch_posts_df(reddit, s, list_type=list_type, limit=limit_per_sub)
    print(f"  → {len(df_s)} posts obtenidos")
    frames.append(df_s)
    time.sleep(0.8)  # pequeña pausa para ser amable con la API

posts = pd.concat(frames, ignore_index=True)
posts_path = OUTDIR / "reddit_posts.csv"
posts.to_csv(posts_path, index=False, encoding="utf-8")

print(f"\n✅ Guardé posts en: {posts_path} (total={len(posts)})")
print("\nDistribución por subreddit:")
display(posts.groupby("subreddit")["id"].count().to_frame("n_posts"))

print("\nVista previa:")
display(posts.head(10))


Descargando 20 posts (hot) de r/politics…
  → 20 posts obtenidos
Descargando 20 posts (hot) de r/PoliticalDiscussion…
  → 20 posts obtenidos
Descargando 20 posts (hot) de r/worldnews…
  → 20 posts obtenidos

✅ Guardé posts en: output\reddit_posts.csv (total=60)

Distribución por subreddit:


,n_posts
subreddit,
PoliticalDiscussion,20
politics,20
worldnews,20



Vista previa:


,subreddit,id,title,score,num_comments,url
0,politics,1n43su4,Saturday Morning Political Cartoon Thread,34,64,https://www.reddit.com/r/politics/comments/1n4...
1,politics,1n53fzg,Fmr. CDC director says you can no longer trust...,5504,121,https://www.msnbc.com/the-weekend-primetime/wa...
2,politics,1n4u384,"Trump, 79, Goes on Bizarre AI Posting Spree",14404,1026,https://www.thedailybeast.com/trump-79-goes-on...
3,politics,1n55zd3,"Already Pardoned by Trump, Jan. 6 Rioters Push...",1739,206,https://www.nytimes.com/2025/08/31/us/politics...
4,politics,1n534ci,Republicans Keep Getting Booed at Their Own To...,2321,71,https://www.motherjones.com/politics/2025/08/r...
5,politics,1n4uee7,Mike Johnson Totally Deflects When Asked About...,7291,176,https://newrepublic.com/post/199857/mike-johns...
6,politics,1n50hky,Red State Cities Under Consideration for Troop...,2354,206,https://www.newsweek.com/dhs-kristi-noem-red-s...
7,politics,1n4stub,Bernie Sanders endorses popular oyster farmer ...,5594,293,https://www.independent.co.uk/news/world/ameri...
8,politics,1n51jd4,Donald Trump is on the brink of becoming a dic...,1399,539,https://www.theglobeandmail.com/opinion/articl...
9,politics,1n569ck,New York's Ultra Rich Are in the Hamptons Frea...,764,127,https://www.rollingstone.com/politics/politics...


## Parte 3: Recolección de comentarios (hasta 5 por post)

**Objetivo:** para cada post descargado anteriormente, recolecto **hasta 5 comentarios** con más puntaje y los guardo en un CSV.

- Fuente: el archivo `output/reddit_posts.csv` generado en el paso anterior.
- Para cada `post_id`:
  - Expando comentarios y filtro los que están vacíos o borrados.
  - Tomo los **5 con mayor `score`** (si hay menos, guardo los que existan).
- Campos por comentario:
  - `post_id` (para enlazar con la tabla de posts)
  - `body` (texto del comentario, sin saltos de línea)
  - `score` (upvotes del comentario)
- Salida: `output/reddit_comments.csv`

> Nota: uso pausas cortas para no golpear la API y manejo errores por post, así no se detiene todo el proceso si uno falla.


In [ ]:
# Paso 4 — Descarga de comentarios (hasta 5 por post) y guardado a CSV

import time
from pathlib import Path
import pandas as pd

# 1) Cargo los posts del paso anterior
OUTDIR = Path("output")
posts_path = OUTDIR / "reddit_posts.csv"
assert posts_path.exists(), "No encuentro output/reddit_posts.csv. Corre primero el Paso 3."

posts = pd.read_csv(posts_path)

print(f"Total de posts cargados: {len(posts)}")
display(posts.head())

# 2) Función para traer comentarios top por post_id
def fetch_top_comments(reddit, post_id: str, per_post: int = 5):
    """
    Devuelve lista de diccionarios con: post_id, body, score.
    Tomo hasta 'per_post' comentarios mejor puntuados.
    """
    subm = reddit.submission(id=post_id)
    subm.comments.replace_more(limit=0)  # expando "MoreComments"
    comments = subm.comments.list()

    # Filtro borrados/vacíos y ordeno por score desc
    clean = []
    for c in comments:
        body = getattr(c, "body", None)
        if body and body not in ("[deleted]", "[removed]"):
            clean.append(c)
    clean.sort(key=lambda c: getattr(c, "score", 0) or 0, reverse=True)

    rows = []
    for c in clean[:per_post]:
        txt = (c.body or "").replace("\n", " ").strip()
        rows.append({"post_id": post_id, "body": txt, "score": int(getattr(c, "score", 0) or 0)})
    return rows

# 3) Itero por todos los posts y voy acumulando
all_comments = []
errors = 0

for i, pid in enumerate(posts["id"].tolist(), start=1):
    try:
        rows = fetch_top_comments(reddit, pid, per_post=5)
        all_comments.extend(rows)
        # Pausa breve para ser amable con la API
        time.sleep(0.5)
    except Exception as e:
        errors += 1
        print(f"⚠️ Error en post {pid}: {e}")
        time.sleep(1.0)

# 4) Armo DataFrame y guardo
comments = pd.DataFrame(all_comments, columns=["post_id", "body", "score"])
comments_path = OUTDIR / "reddit_comments.csv"
comments.to_csv(comments_path, index=False, encoding="utf-8")

print(f"\n✅ Guardé comentarios en: {comments_path} (total={len(comments)})")
print(f"ℹ️ Posts con error: {errors}")

# 5) Chequeos rápidos de integridad
print("\nEjemplos (5 filas):")
display(comments.head())

print("\nCantidad de comentarios por post (top 10):")
display(comments.groupby("post_id")["score"].count().sort_values(ascending=False).head(10).to_frame("n_comments"))


Total de posts cargados: 60


,subreddit,id,title,score,num_comments,url
0,politics,1n43su4,Saturday Morning Political Cartoon Thread,34,64,https://www.reddit.com/r/politics/comments/1n4...
1,politics,1n53fzg,Fmr. CDC director says you can no longer trust...,5504,121,https://www.msnbc.com/the-weekend-primetime/wa...
2,politics,1n4u384,"Trump, 79, Goes on Bizarre AI Posting Spree",14404,1026,https://www.thedailybeast.com/trump-79-goes-on...
3,politics,1n55zd3,"Already Pardoned by Trump, Jan. 6 Rioters Push...",1739,206,https://www.nytimes.com/2025/08/31/us/politics...
4,politics,1n534ci,Republicans Keep Getting Booed at Their Own To...,2321,71,https://www.motherjones.com/politics/2025/08/r...



✅ Guardé comentarios en: output\reddit_comments.csv (total=300)
ℹ️ Posts con error: 0

Ejemplos (5 filas):


,post_id,body,score
0,1n43su4,Jen Sorensen: [Web’s end]( https://www.daily...,29
1,1n43su4,Pedro X. Molina: [The symbol…](https://www.g...,24
2,1n43su4,Brian McFadden - [Anti-Heroes](https://www.dai...,20
3,1n43su4,Ben Jennings [Back to school sale](https://w...,19
4,1n43su4,Ratt- [Who is to blame?](https://www.dailycart...,17



Cantidad de comentarios por post (top 10):


,n_comments
post_id,
1bwbuka,5
1msce9l,5
1mxwbnt,5
1myv6j9,5
1mz6m1w,5
1mzn0u5,5
1mzt9qg,5
1mzv1a2,5
1n09ltj,5


### Resultado de la recolección de comentarios
- Archivo generado: `output/reddit_comments.csv`
- Claves:
  - `post_id` me permite enlazar con `reddit_posts.csv`.
  - `score` ayuda a priorizar comentarios más relevantes.
  - `body` es el texto que luego usaré para análisis de sentimiento.
- Con esto cierro la recolección de datos (posts + comentarios) y quedo lista para la fase de **análisis exploratorio / sentiment analysis**.


## Parte final: Verificación del enlace post–comentario + Mini-EDA

**Objetivo:**  
- Verifico que cada comentario (`reddit_comments.csv`) está correctamente vinculado a su post padre (`reddit_posts.csv`) mediante `post_id`.  
- Hago un análisis rápido (EDA) para agregar valor:
  - Top posts por `score` (upvotes).
  - Posts con más comentarios (según `num_comments`).
  - Comentarios por subreddit.
  - Top posts por “impacto de comentarios” (suma de `score` de los comentarios).

Además, genero un archivo `output/reddit_merged_sample.csv` con ejemplo de unión *post + comentario* para evidenciar el enlace.


In [ ]:
# Verifico el enlace post–comentario y hago un pequeño EDA

import pandas as pd
from pathlib import Path

OUTDIR = Path("output")
posts_path = OUTDIR / "reddit_posts.csv"
comments_path = OUTDIR / "reddit_comments.csv"

# 1) Cargo las tablas generadas
df_posts = pd.read_csv(posts_path)
df_comments = pd.read_csv(comments_path)

print(f"Posts: {len(df_posts)} filas | Comments: {len(df_comments)} filas")

# 2) Verificación: uno comentarios con su post por el id
merged = df_comments.merge(
    df_posts[["id", "title", "subreddit", "score", "num_comments", "url"]],
    left_on="post_id",
    right_on="id",
    how="left",
    validate="many_to_one"  # cada comment pertenece a un único post
)

# Reviso integridad del link (cuántos comentarios quedaron sin post)
missing = merged["title"].isna().sum()
print(f"Comentarios sin post emparejado: {missing}")

# 3) Exporto una muestra como evidencia del link post–comentario
sample_cols = ["post_id", "subreddit", "title", "comment_body", "comment_score"]
# Si tus columnas se llaman "body" y "score" en comments, ajusto:
if "comment_body" not in merged.columns:
    merged = merged.rename(columns={"body": "comment_body"})
if "comment_score" not in merged.columns:
    merged = merged.rename(columns={"score_x": "comment_score"})  # por si hay colisión de nombre
    if "comment_score" not in merged.columns and "score" in merged.columns:
        merged = merged.rename(columns={"score": "comment_score"})

evidence = merged[sample_cols].head(50)
evidence_path = OUTDIR / "reddit_merged_sample.csv"
evidence.to_csv(evidence_path, index=False, encoding="utf-8")
print(f"✅ Evidencia guardada: {evidence_path}")

display(evidence.head(10))

# 4) Mini-EDA que agrega valor
print("\nTop 10 posts por score (upvotes):")
top_by_score = df_posts.sort_values("score", ascending=False).head(10)[["subreddit", "title", "score", "num_comments", "url"]]
display(top_by_score)

print("\nTop 10 posts por número de comentarios (según metadata del post):")
top_by_num_comments = df_posts.sort_values("num_comments", ascending=False).head(10)[["subreddit", "title", "num_comments", "score", "url"]]
display(top_by_num_comments)

print("\nComentarios por subreddit (conteo) según el dataset de comentarios:")
comments_per_sub = merged.groupby("subreddit")["comment_body"].count().sort_values(ascending=False).to_frame("n_comments")
display(comments_per_sub)

print("\nTop 10 posts por 'impacto de comentarios' (suma de score de comentarios):")
impact = merged.groupby(["post_id", "title", "subreddit"], dropna=False)["comment_score"].sum().reset_index()
impact = impact.sort_values("comment_score", ascending=False).head(10)
display(impact)


Posts: 60 filas | Comments: 300 filas
Comentarios sin post emparejado: 0
✅ Evidencia guardada: output\reddit_merged_sample.csv


,post_id,subreddit,title,comment_body,comment_score
0,1n43su4,politics,Saturday Morning Political Cartoon Thread,Jen Sorensen: [Web’s end]( https://www.daily...,29
1,1n43su4,politics,Saturday Morning Political Cartoon Thread,Pedro X. Molina: [The symbol…](https://www.g...,24
2,1n43su4,politics,Saturday Morning Political Cartoon Thread,Brian McFadden - [Anti-Heroes](https://www.dai...,20
3,1n43su4,politics,Saturday Morning Political Cartoon Thread,Ben Jennings [Back to school sale](https://w...,19
4,1n43su4,politics,Saturday Morning Political Cartoon Thread,Ratt- [Who is to blame?](https://www.dailycart...,17
5,1n53fzg,politics,Fmr. CDC director says you can no longer trust...,"As a reminder, this subreddit [is for civil di...",1
6,1n53fzg,politics,Fmr. CDC director says you can no longer trust...,Everything's become an opinion piece from cons...,1
7,1n53fzg,politics,Fmr. CDC director says you can no longer trust...,Just in case any Americans are looking for rel...,1
8,1n53fzg,politics,Fmr. CDC director says you can no longer trust...,I don’t trust any agency under Trump’s control...,1
9,1n53fzg,politics,Fmr. CDC director says you can no longer trust...,Step 1 : Train your populace to not trust inst...,1



Top 10 posts por score (upvotes):


,subreddit,title,score,num_comments,url
59,worldnews,Company behind Jack Daniel's says Canadian boy...,18658,881,https://www.cbc.ca/news/canada/nova-scotia/bro...
2,politics,"Trump, 79, Goes on Bizarre AI Posting Spree",14404,1026,https://www.thedailybeast.com/trump-79-goes-on...
44,worldnews,Russia’s 2025 Summer Offensive Ends in Heavy L...,12682,456,https://united24media.com/latest-news/russias-...
41,worldnews,"US spies stoked separatism in Greenland, Denma...",10464,492,https://euobserver.com/eu-and-the-world/ar4837...
18,politics,Trump's distractions fail as Congress is set t...,9093,106,https://www.msnbc.com/the-briefing-with-jen-ps...
5,politics,Mike Johnson Totally Deflects When Asked About...,7291,176,https://newrepublic.com/post/199857/mike-johns...
43,worldnews,Venezuela warns US to stay away from national ...,6055,423,https://www.trtworld.com/world/article/2cd488f...
7,politics,Bernie Sanders endorses popular oyster farmer ...,5594,293,https://www.independent.co.uk/news/world/ameri...
1,politics,Fmr. CDC director says you can no longer trust...,5504,121,https://www.msnbc.com/the-weekend-primetime/wa...
52,worldnews,Argentina: Milei government panics after fresh...,4933,204,https://batimes.com.ar/news/argentina/milei-go...



Top 10 posts por número de comentarios (según metadata del post):


,subreddit,title,num_comments,score,url
20,PoliticalDiscussion,Casual Questions Thread,8308,88,https://www.reddit.com/r/PoliticalDiscussion/c...
2,politics,"Trump, 79, Goes on Bizarre AI Posting Spree",1026,14404,https://www.thedailybeast.com/trump-79-goes-on...
54,worldnews,Aid flotilla with Greta Thunberg set to sail f...,1025,3180,https://www.theguardian.com/environment/2025/a...
59,worldnews,Company behind Jack Daniel's says Canadian boy...,881,18658,https://www.cbc.ca/news/canada/nova-scotia/bro...
8,politics,Donald Trump is on the brink of becoming a dic...,539,1399,https://www.theglobeandmail.com/opinion/articl...
41,worldnews,"US spies stoked separatism in Greenland, Denma...",492,10464,https://euobserver.com/eu-and-the-world/ar4837...
23,PoliticalDiscussion,What do you think about Gavin Newsom's new soc...,489,745,https://www.reddit.com/r/PoliticalDiscussion/c...
44,worldnews,Russia’s 2025 Summer Offensive Ends in Heavy L...,456,12682,https://united24media.com/latest-news/russias-...
43,worldnews,Venezuela warns US to stay away from national ...,423,6055,https://www.trtworld.com/world/article/2cd488f...
19,politics,"Blocking tariffs would be ""end of the United S...",351,626,https://www.axios.com/2025/08/31/trump-tariffs...



Comentarios por subreddit (conteo) según el dataset de comentarios:


,n_comments
subreddit,
PoliticalDiscussion,100
politics,100
worldnews,100



Top 10 posts por 'impacto de comentarios' (suma de score de comentarios):


,post_id,title,subreddit,comment_score
34,1n4u384,"Trump, 79, Goes on Bizarre AI Posting Spree",politics,23283
21,1n4amb2,Company behind Jack Daniel's says Canadian boy...,worldnews,12388
25,1n4lyl2,Aid flotilla with Greta Thunberg set to sail f...,worldnews,8441
55,1n55kzl,"US spies stoked separatism in Greenland, Denma...",worldnews,7468
33,1n4ty5j,Venezuela warns US to stay away from national ...,worldnews,5637
52,1n54p1m,Trump Officials Discussed $500M Alberta (Canad...,worldnews,5553
22,1n4jkim,Argentina: Milei government panics after fresh...,worldnews,4221
27,1n4qqg1,Russia’s 2025 Summer Offensive Ends in Heavy L...,worldnews,4096
31,1n4stub,Bernie Sanders endorses popular oyster farmer ...,politics,3298
35,1n4uee7,Mike Johnson Totally Deflects When Asked About...,politics,2890


# Conclusiones — Ejercicio 2: Reddit API & Sentiment Data Collection

- Se logró **conectar exitosamente a la API de Reddit (PRAW)** usando credenciales seguras guardadas en `.env`.
- Se recolectaron **20 posts por cada subreddit** (`r/politics`, `r/PoliticalDiscussion`, `r/worldnews`), guardando título, score, n_comments, id y url.
- Se descargaron hasta **5 comentarios principales por post**, enlazándolos a su publicación mediante `post_id`.
- Los datos quedaron almacenados en dos CSV (`reddit_posts.csv` y `reddit_comments.csv`) y además se verificó la relación posts–comentarios con un `merge`.

### Insights del mini-EDA:
- Los posts con mayor puntaje (upvotes) y con más comentarios no siempre coinciden, lo que refleja diferencias entre “popularidad” y “participación”.
- `r/politics` y `r/worldnews` concentran la mayoría de los comentarios en el dataset, confirmando su mayor nivel de interacción frente a `r/PoliticalDiscussion`.
- Algunos posts destacan por su **alto “impacto de comentarios”** (suma de score de los comentarios), lo que permite detectar hilos donde la discusión generó mayor resonancia.
- Estos datos recolectados pueden servir como base para un análisis de **sentimiento** en los comentarios (positivo/negativo), enriqueciendo el entendimiento de la polarización política.

---

📌 **Con esto se cumple el Ejercicio 2:** se conectó la API, se almacenaron posts y comentarios vinculados, y se realizó un EDA preliminar que añade valor.
